In [1]:
%load_ext dotenv
# Use find_dotenv to locate the file
%dotenv

In [4]:
%reload_ext dotenv
# %dotenv -o .env.prod
%dotenv -o

In [2]:
import os
from google.cloud import container_v1, dns
from googleapiclient import discovery
import pydash

In [3]:
client = container_v1.ClusterManagerClient()

project_id = 'zac-chung-dev'
zone = 'asia-east1-a'
cluster_id = 'workshop-knative'

response = client.get_cluster(project_id, zone, cluster_id)

In [21]:
client.get_node_pool(project_id, zone, cluster_id, response.node_pools[0].name)

name: "default-pool"
config {
  machine_type: "n1-standard-4"
  disk_size_gb: 30
  oauth_scopes: "https://www.googleapis.com/auth/cloud-platform"
  metadata {
    key: "disable-legacy-endpoints"
    value: "true"
  }
  image_type: "COS"
  service_account: "default"
  preemptible: true
  disk_type: "pd-standard"
}
initial_node_count: 1
autoscaling {
  enabled: true
  min_node_count: 1
  max_node_count: 3
}
management {
  auto_repair: true
}
self_link: "https://container.googleapis.com/v1/projects/zac-chung-dev/zones/asia-east1-a/clusters/workshop-knative/nodePools/default-pool"
version: "1.14.6-gke.1"
instance_group_urls: "https://www.googleapis.com/compute/v1/projects/zac-chung-dev/zones/asia-east1-a/instanceGroupManagers/gke-workshop-knative-default-pool-825bb779-grp"
status: RUNNING

In [17]:
client.create_node_pool(project_id, zone, cluster_id, {
    'name': 'default-pool',
    'config': {
        'machine_type': 'n1-standard-4',
        'disk_size_gb': 30,
        'preemptible': True,
        'oauth_scopes': ('https://www.googleapis.com/auth/cloud-platform',)
    },
    'initial_node_count': 1,
    'autoscaling': {
        'enabled': True,
        'min_node_count': 1,
        'max_node_count': 3,
    },
    'management': {
        'auto_repair': True,
    },
})

name: "operation-1570377090645-35dbda31"
zone: "asia-east1-a"
operation_type: CREATE_NODE_POOL
status: RUNNING
self_link: "https://container.googleapis.com/v1/projects/347397173885/zones/asia-east1-a/operations/operation-1570377090645-35dbda31"
target_link: "https://container.googleapis.com/v1/projects/347397173885/zones/asia-east1-a/clusters/workshop-knative/nodePools/default-pool"
start_time: "2019-10-06T15:51:30.645998157Z"

In [10]:
client.delete_node_pool(project_id, zone, cluster_id, 'default-pool')

name: "operation-1570376161087-f370c458"
zone: "asia-east1-a"
operation_type: DELETE_NODE_POOL
status: RUNNING
self_link: "https://container.googleapis.com/v1/projects/347397173885/zones/asia-east1-a/operations/operation-1570376161087-f370c458"
target_link: "https://container.googleapis.com/v1/projects/347397173885/zones/asia-east1-a/clusters/workshop-knative/nodePools/default-pool"
start_time: "2019-10-06T15:36:01.087878148Z"

In [14]:
api = compute.networks()

In [46]:
dns_client = dns.Client(project=project_id)

In [79]:
aa = dns_client.list_zones()

In [86]:
mz = dns_client.zone(name='base1618-dev', dns_name='base1618-dev.com.')

In [85]:
mz.name, mz.dns_name

('base1618-dev', 'base1618-dev.com.')

In [90]:
mz.create?

In [106]:
mz_rs = list(mz.list_resource_record_sets())

In [107]:
[row.name for row in mz_rs]

['base1618-dev.com.',
 'base1618-dev.com.',
 '*.default.knative.base1618-dev.com.']

In [76]:
bb = mz_rs[2]

In [103]:
[row.__dict__ for row in mz_rs]

[{'name': 'base1618-dev.com.',
  'record_type': 'NS',
  'ttl': 21600,
  'rrdatas': ['ns-cloud-a1.googledomains.com.',
   'ns-cloud-a2.googledomains.com.',
   'ns-cloud-a3.googledomains.com.',
   'ns-cloud-a4.googledomains.com.'],
  'zone': <google.cloud.dns.zone.ManagedZone at 0x10f0b06d0>},
 {'name': 'base1618-dev.com.',
  'record_type': 'SOA',
  'ttl': 21600,
  'rrdatas': ['ns-cloud-a1.googledomains.com. cloud-dns-hostmaster.google.com. 1 21600 3600 259200 300'],
  'zone': <google.cloud.dns.zone.ManagedZone at 0x10f0b06d0>},
 {'name': '*.default.knative.base1618-dev.com.',
  'record_type': 'A',
  'ttl': 300,
  'rrdatas': ['35.185.132.252'],
  'zone': <google.cloud.dns.zone.ManagedZone at 0x10f0b06d0>}]

In [100]:
[row.record_type for row in mz_rs]

['NS', 'SOA', 'A']

In [94]:
dns_change = mz.changes()